In [9]:
import nltk
from gensim.models import Word2Vec, KeyedVectors
from gensim.test.utils import common_texts
from sklearn import linear_model
from train import tokenize_sents, tokenize_text

import numpy as np

# reg = linear_model.LogisticRegression()


# target => array?
# model takes n vectors of the previous words in a sentence and gives the probability for each combination 



In [10]:
from collections import defaultdict
import pandas as pd
from sklearn import linear_model, ensemble
from train import Ngramm
from train import tokenize_sents, tokenize_text
import pickle

# reg.predict([[context, word_POS, ]])

ngramm_count = defaultdict(lambda: 0)
ngramm_words = defaultdict(lambda: defaultdict(lambda: 0))

words = set()

with open("data\\catcher_in_the_rye.txt", "r", encoding="UTF-8") as fl:
# with open("data\\harry_potter.txt", "r", encoding="UTF-8") as fl:
    text = fl.read()

sents = [tokenize_sents(x) for x in tokenize_text(text)]
len(sents)


58494

In [11]:
n = 2

for sentence in sents:
    for context, word in Ngramm.ngramms(sentence, n):
        ngramm_words[context][word] += 1
        ngramm_count[context] += 1
        words.add(word)
        

model = Word2Vec(sents, min_count=1)

pos_lst = nltk.pos_tag_sents(sents)
        
X = []
y = []

for i, sentence in enumerate(sents):
    for j, (context, word) in enumerate(Ngramm.ngramms(sentence, n)):
        prob = ngramm_words[context][word] / ngramm_count[context]
        vec = model.wv[word]
        vec1 = model.wv[context[0]]
        vec2 = model.wv[context[1]]
        pos = pos_lst[i][j + n][1]
        X.append([*vec1, *vec2, *vec])
        y.append(prob)
        # print(prob)

reg = ensemble.BaggingRegressor()
reg.fit(X, y)

with open("reg.pkl", "wb") as fl:
    pickle.dump(reg, fl)
    



In [12]:
with open("reg.pkl", "rb") as fl:
    reg = pickle.load(fl)

with open("D:\\practice\\tinkoff_ml\\data\\harry_potter.txt", "r", encoding="UTF-8") as fl:
    text = fl.read()

sents = [tokenize_sents(x) for x in tokenize_text(text)]

model = Word2Vec(sents, min_count=1)

In [26]:
reg.predict([[*model.wv["i"], *model.wv["will"], *model.wv["check"]]])

array([0.61916667])